In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import category_encoders
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
import matplotlib as plt
from random import randint

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder



print("Setup Complete")

Setup Complete


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Read the file into a variable fifa_data
filepath = "/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Train.xlsx"
data = pd.read_excel(filepath)


In [3]:
# Read the file into a variable fifa_data
 
test = pd.read_excel("/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Test.xlsx")


In [4]:
submission= pd.read_excel('/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Sample_Submission.xlsx')
# data.head(3)

In [5]:
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)
    
    
    
    
    
# from sklearn.tree import DecisionTreeClassifier

# from sklearn.ensemble import BaggingClassifier

# clf1 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
#                        max_depth=None, max_features=11, max_leaf_nodes=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=8, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, presort='deprecated',
#                        random_state=None, splitter='best')

# clf2 = BaggingClassifier(base_estimator=clf1, n_estimators=100, max_samples=1.0, max_features=1.0, 
#                          bootstrap=True,
#                          bootstrap_features=False, oob_score=True, warm_start=False,
#                          n_jobs=-1, random_state=None, verbose=0)

# clf3=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
#               learning_rate=0.1, max_delta_step=0, max_depth=5,
#               min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
#               nthread=None, objective='multi:softprob', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=0.6, verbosity=1)  

In [6]:
def dropcol(train,test):
    train['logID'] = np.log1p(train['ID'])
    test['logID'] = np.log1p(test['ID'])
    cols_to_drop = ['Date','ID']
    dtrain = train.drop(cols_to_drop,axis =1)
    dtest = test.drop(cols_to_drop,axis =1)
    
    print(dtrain.shape,dtest.shape)

    return dtrain,dtest

def catVar1(data):
    categorical_colsT1 = [cname for cname in data.columns if
                    data[cname].nunique() <=10 and 
                    data[cname].dtype == "object"]
    return categorical_colsT1

def catVar2(data):
    categorical_colsT2 = [cname for cname in data.columns if
                    data[cname].nunique() >10 and 
                    data[cname].dtype == "object"]
    return categorical_colsT2

def NumVar(data) :
    numerical_cols = [cname for cname in data.columns if 
    data[cname].dtype in ['int64', 'float64']]
    return numerical_cols

def imputer(train,test):
    
    from sklearn.impute import SimpleImputer
    
    numerical_cols = NumVar(train)
    my_imputer = SimpleImputer(strategy='most_frequent')
    imputed_train = pd.DataFrame(my_imputer.fit_transform(train))
    imputed_test = pd.DataFrame(my_imputer.transform(test))

    # Imputation removed column names; put them back
    imputed_train.columns = train.columns
    imputed_test.columns = test.columns
    
    #restoring datatypes 
    imputed_train[numerical_cols] = imputed_train[numerical_cols].apply(pd.to_numeric, errors='coerce')
    imputed_test[numerical_cols] = imputed_test[numerical_cols].apply(pd.to_numeric, errors='coerce')
    print(imputed_train.shape,imputed_test.shape)
    return imputed_train,imputed_test 

def robustlabelencoder(train,test):
    from sklearn.preprocessing import LabelEncoder

    label_enc = LabelEncoderExt()
    cols = catVar2(train)
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        train[col] = label_enc.transform(train[col])
        test[col] = label_enc.transform(test[col])
        
    print(train.shape,test.shape)
    
    return train,test

def normalabelencoder(train,test,cols):
    from sklearn.preprocessing import LabelEncoder

    label_enc = LabelEncoder()
    
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        train[col] = label_enc.transform(train[col])
        test[col] = label_enc.transform(test[col])
    
    return train,test



def ohebygetdummis(train,test):
    pass

def onhotencoder(train,test):
    
    from sklearn.preprocessing import OneHotEncoder
    object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

def missingcheck(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
    return missing_data

def targetencoding(train,test,y_train):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = catVar2(train)
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
    testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc


def special(train,test):
    train['Geo_Loc'] = train['Geo_Loc'].str.replace(r'\D', '')
    train['Geo_Loc'] = pd.to_numeric(train['Geo_Loc'], errors='coerce') 
    
    test['Geo_Loc'] = test['Geo_Loc'].str.replace(r'\D', '')
    test['Geo_Loc'] = pd.to_numeric(test['Geo_Loc'], errors='coerce')
    
    train['Date'] = pd.to_datetime(train['Date'] ,errors='coerce')
    test['Date'] = pd.to_datetime(test['Date'] ,errors='coerce')
    
    train = train.assign(
               hour=train.Date.dt.hour,
               day=train.Date.dt.day,
               month=train.Date.dt.month,
               year=train.Date.dt.year
                        )
    
    test = test.assign(
               hour=test.Date.dt.hour,
               day=test.Date.dt.day,
               month=test.Date.dt.month,
               year=test.Date.dt.year
                        )
    print(train.shape,test.shape)
    
    return train, test

In [7]:
# def syntheticvariable(train , y ):
#     from imblearn.over_sampling import SMOTENC
#     from collections import Counter
#     categorical_features = [cname.index for cname in train.columns if train[cname].dtype == "object"]
#     out = np.argwhere(train.columns.isin(categorical_features)).ravel().tolist()
    
#     smote_nc = SMOTENC([3, 5, 6, 8, 10], random_state=0)  #instead of passing out i need to hardcode it.

#     X_resampled, y_resampled = smote_nc.fit_resample(train,y)
    
#     print(sorted(Counter(y_resampled).items()))
    
#     return X_resampled,y_resampled
    

In [8]:
#####crete new features###

In [9]:
def createIntercations(train,test,cat_features):
    import itertools
    print(f"Creating features on {cat_features}, with combination 2 for training data /n")
    interactionstrain = pd.DataFrame(index=train.index)
    
    for col1 ,col2 in  itertools.combinations(cat_features,2):   
        newcolname = col1 + "_" + col2 
        new_values = train[col1].map(str) + "_" + train[col2].map(str)
        interactionstrain[newcolname] = new_values

    
    train_df = train.join(interactionstrain)
    
    print(f"Creating features on {cat_features}, with combination 2 for testing data")
    interactionstest = pd.DataFrame(index=train.index)
    
    for col1 ,col2 in  itertools.combinations(cat_features,2):   
        newcolname = col1 + "_" + col2 
        new_values = test[col1].map(str) + "_" + test[col2].map(str)
        interactionstest[newcolname] = new_values

    test_df = test.join(interactionstest)
    
    print(train_df.shape,test_df.shape)
    
    
    return train_df,test_df


In [10]:
def createmeanfestures(train,test ,features):
  
  cols = features
  interactionstrain = pd.DataFrame(index=train.index)
  interactionstest = pd.DataFrame(index=test.index)

  for col  in cols:   
        newcolname = col + "_mean"   
        new_values = train[col].mean()
        interactionstrain[newcolname] = new_values

  for col  in cols:   
        newcolname = col + "_mean"   
        new_values = test[col].mean()
        interactionstest[newcolname] = new_values


  train_df = train.join(interactionstrain)
  test_df = test.join(interactionstest)

  print(train_df.shape,test_df.shape)

  return train_df,test_df




def createlogfestures(train,test,features):
  
  cols = features
  interactionstrain = pd.DataFrame(index=train.index)
  interactionstest = pd.DataFrame(index=test.index)
  for col  in cols:   
        newcolname = col + "_log"   
        new_values = np.log1p(train[col])
        interactionstrain[newcolname] = new_values

  for col  in cols:   
        newcolname = col + "_log"   
        new_values = np.log1p(test[col])
        interactionstest[newcolname] = new_values


  train_df = train.join(interactionstrain)
  test_df = test.join(interactionstest)

  print(train_df.shape,test_df.shape)


  return train_df ,test_df

def createsqrtfeatures(train,test,features):
    

  cols = features
  interactionstrain = pd.DataFrame(index=train.index)
  interactionstest = pd.DataFrame(index=test.index)
  
  for col  in cols:   
        newcolname = col + "_sqrt"   
        new_values = np.sqrt(train[col])
        interactionstrain[newcolname] = new_values

  for col  in cols:   
        newcolname = col + "_sqrt"   
        new_values = np.sqrt(test[col])
        interactionstest[newcolname] = new_values


  train_df = train.join(interactionstrain)
  test_df = test.join(interactionstest)

  print(train_df.shape,test_df.shape)


  return train_df ,test_df


In [11]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
    
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    #model = RandomForestClassifier(random_state=0)
    #model.fit(X_train, y_train)
    from sklearn.tree import DecisionTreeClassifier

    from sklearn.ensemble import BaggingClassifier

    clf1 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=25, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

    clf2 = BaggingClassifier(base_estimator=clf1, n_estimators=10, 
                             bootstrap=True,
                             bootstrap_features=False, oob_score=True, warm_start=False,
                             n_jobs=-1, random_state=786, verbose=0)
    clf2.fit(X_train,y_train)

    preds = clf2.predict(X_valid)
    target_names = ['class 0', 'class 1', 'class 2','class 3', 'class 4', 'class 5', 'class 6']
    print(classification_report(y_valid, preds, target_names=target_names,labels= [0,1,2,3,4,5,6]))
    return clf2.score(X_valid,y_valid) 

In [12]:
# X_train, X_valid, y_train, y_valid
#clf1

In [37]:
y = data.Inspection_Results

# To keep things simple, we'll use only numerical predictors
train = data.drop(['Inspection_Results'], axis=1)


In [38]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.80, test_size=0.20,random_state=0)

In [39]:
trainS, testS = special(X_train ,X_valid)

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

(117954, 18) (29489, 18)


In [40]:
trainSd, testSd = dropcol(trainS,testS)

(117954, 17) (29489, 17)


In [41]:
trainSdIm, testSdIm = imputer(trainSd, testSd)

(117954, 17) (29489, 17)


In [42]:
missingcheck(trainSdIm),missingcheck(testSdIm) 

(                   Total    %
 logID                  0  0.0
 LocationID             0  0.0
 FacilityID             0  0.0
 FacilityName           0  0.0
 Type                   0  0.0
 Street                 0  0.0
 City                   0  0.0
 State                  0  0.0
 Reason                 0  0.0
 year                   0  0.0
 SectionViolations      0  0.0
 RiskLevel              0  0.0
 Geo_Loc                0  0.0
 hour                   0  0.0
 day                    0  0.0
 month                  0  0.0
 LicenseNo              0  0.0,                    Total    %
 logID                  0  0.0
 LocationID             0  0.0
 FacilityID             0  0.0
 FacilityName           0  0.0
 Type                   0  0.0
 Street                 0  0.0
 City                   0  0.0
 State                  0  0.0
 Reason                 0  0.0
 year                   0  0.0
 SectionViolations      0  0.0
 RiskLevel              0  0.0
 Geo_Loc                0  0.0
 hour   

In [43]:
trainSdImIn1,testSdImIn1 = createIntercations(trainSdIm, testSdIm,cat_features = ['SectionViolations','RiskLevel','Reason'])

Creating features on ['SectionViolations', 'RiskLevel', 'Reason'], with combination 2 for training data /n
Creating features on ['SectionViolations', 'RiskLevel', 'Reason'], with combination 2 for testing data
(117954, 20) (29489, 20)


In [44]:
trainSdImIn12,testSdImIn12 = createIntercations(trainSdImIn1,testSdImIn1,cat_features = ['State','City','Street','LocationID','Geo_Loc']
)

Creating features on ['State', 'City', 'Street', 'LocationID', 'Geo_Loc'], with combination 2 for training data /n
Creating features on ['State', 'City', 'Street', 'LocationID', 'Geo_Loc'], with combination 2 for testing data
(117954, 30) (29489, 30)


In [45]:
# X_trainSDrIMintertgenc,X_validSDrIMinter2tgenc = targetencoding(X_trainSDrIMinter2,X_validSDrIMinter2,y_train)

In [57]:
trainSdImIn12lb,testSdImIn12lb = robustlabelencoder(trainSdImIn12,testSdImIn12 )

['Type', 'Reason', 'SectionViolations_RiskLevel', 'SectionViolations_Reason', 'RiskLevel_Reason', 'State_Street', 'State_LocationID', 'State_Geo_Loc', 'City_Street', 'City_LocationID', 'City_Geo_Loc', 'Street_LocationID', 'Street_Geo_Loc', 'LocationID_Geo_Loc']
(117954, 30) (29489, 30)


In [58]:
trainSdImIn12lbohe,testSdImIn12lbohe = onhotencoder(trainSdImIn12lb,testSdImIn12lb)

['City', 'State', 'RiskLevel', 'State_City']
(117954, 38) (29489, 38)


In [59]:
features = trainSdImIn12lbohe.columns
features

Index(['LicenseNo', 'FacilityID', 'FacilityName', 'Type', 'Street',
       'LocationID', 'Reason', 'SectionViolations', 'Geo_Loc', 'hour', 'day',
       'month', 'year', 'logID', 'SectionViolations_RiskLevel',
       'SectionViolations_Reason', 'RiskLevel_Reason', 'State_Street',
       'State_LocationID', 'State_Geo_Loc', 'City_Street', 'City_LocationID',
       'City_Geo_Loc', 'Street_LocationID', 'Street_Geo_Loc',
       'LocationID_Geo_Loc', 'City_id-11235901', 'City_id-11275913',
       'State_id_1890134', 'State_id_1890135', 'RiskLevel_High',
       'RiskLevel_Low', 'RiskLevel_Medium', 'RiskLevel_Uncertain',
       'State_City_id_1890134_id-11235901',
       'State_City_id_1890134_id-11275913',
       'State_City_id_1890135_id-11235901',
       'State_City_id_1890135_id-11275913'],
      dtype='object')

In [60]:
# featres = ['LicenseNo',
#            'FacilityID', 
#            'FacilityName', 
#            'Type', 
#            'Street',
#            'LocationID', 
#            'Reason', 
#            'SectionViolations', 
#            'Geo_Loc',
#            'hour', 
#            'day',
#            'month', 
#            'year', 
            
#            'SectionViolations_RiskLevel',
#            'SectionViolations_Reason',
#            'RiskLevel_Reason', 
#            'State_Street',
#            'State_LocationID', 
#            'State_Geo_Loc',
#            'City_Street', 
#            'City_LocationID',
#            'City_Geo_Loc', 
#            'Street_LocationID', 
#            'Street_Geo_Loc',
#            'LocationID_Geo_Loc', 
#            'City_id-11235901',
#            'City_id-11275913',
#            'State_id_1890134', 
#            'State_id_1890135', 
#            'RiskLevel_High',
#            'RiskLevel_Low',
#            'RiskLevel_Medium',
#            'RiskLevel_Uncertain',
#                      ]

In [61]:
# 'State_City_id_1890134_id-11235901',
#            'State_City_id_1890134_id-11275913',
#            'State_City_id_1890135_id-11235901',
#            'State_City_id_1890135_id-11275913'
# drop these

In [62]:
def dropcol2(train,test):
    cols_to_drop =['hour',
                   'State_City_id_1890134_id-11235901',
                   'State_City_id_1890134_id-11275913',
                   'State_City_id_1890135_id-11235901',
                   'State_City_id_1890135_id-11275913'
]
    dtrain = train.drop(cols_to_drop,axis =1)
    dtest = test.drop(cols_to_drop,axis =1)
    
    print(dtrain.shape,dtest.shape)

    return dtrain,dtest

In [63]:
trainSdImIn12lboheD,testSdImIn12lboheD = dropcol2(trainSdImIn12lbohe,testSdImIn12lbohe )

(117954, 33) (29489, 33)


In [64]:
score_dataset(trainSdImIn12lboheD,testSdImIn12lboheD, y_train, y_valid)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_bagging.py:638: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00         9
     class 1       0.79      0.70      0.74      5747
     class 2       0.45      0.31      0.36       258
     class 3       0.45      0.20      0.28       909
     class 4       0.88      0.94      0.91     15911
     class 5       0.70      0.75      0.72      4074
     class 6       0.65      0.66      0.66      2581

    accuracy                           0.81     29489
   macro avg       0.56      0.51      0.52     29489
weighted avg       0.80      0.81      0.80     29489



0.8109464546101937

In [29]:
#trainSdImIn12lboheLog,testSdImIn12lboheLog = createlogfestures(trainSdImIn12lbohe,testSdImIn12lbohe,features)

In [30]:
# trainSdImIn12lbohe,testSdImIn12lbohe = createsqrtfeatures(trainSdImIn12lbohe,testSdImIn12lbohe,features)

In [68]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from lightgbm import LGBMClassifier
clf1 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=33, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

clf2 = BaggingClassifier(base_estimator=clf1, n_estimators=100, 
                             bootstrap=True,
                             bootstrap_features=False, oob_score=True, warm_start=False,
                             n_jobs=-1, random_state=786, verbose=0)

# clf3=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
#               learning_rate=0.1, max_delta_step=0, max_depth=5,
#               min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
#               nthread=None, objective='multi:softprob', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=0.6, verbosity=1)


clf4 = LGBMClassifier(n_estimators=5000, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
    

In [69]:
clf5=LGBMClassifier(n_estimators = 10000, 
    learning_rate= 0.015,
    boosting_type = 'gbdt', 
    colsample_bytree= 0.80, 
    min_child_weight= 10.0, 
    num_leaves = 64, 
    objective='multiclass', 
    num_class= 7,
    subsample = 0.70, 
    subsample_freq= 5,     
    n_jobs=-1)

In [70]:
clf6=LGBMClassifier(n_estimators = 15000, 
    learning_rate= 0.01,
    boosting_type = 'gbdt', 
    colsample_bytree= 0.80, 
    min_child_weight= 9.0, 
    num_leaves = 64, 
    objective='multiclass', 
    num_class= 7,
    subsample = 0.85, 
    subsample_freq= 6,     
    n_jobs=-1)

In [ ]:
clf6.fit(trainSdImIn12lboheD,y_train)
preds = clf2.predict(testSdImIn12lboheD)
target_names = ['class 0', 'class 1', 'class 2','class 3', 'class 4', 'class 5', 'class 6']
print(classification_report(y_valid, preds, target_names=target_names,labels= [0,1,2,3,4,5,6]))
    